# 导入包

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

# 加载数据集

In [2]:
# 读取训练集和测试集
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

# 数据清洗

In [ ]:
# plt.scatter(train_df['房屋面积'], train_df['月租金'], c = "blue", marker = "s")
# plt.title("Looking for outliers")
# plt.xlabel("RoomArea")
# plt.ylabel("Rental")
# plt.show()

In [3]:
# 异常值清洗,从1.94576上升到1.90965
train_df = train_df.drop(train_df[(train_df['房屋面积']>0.1)].index)

In [ ]:
# plt.scatter(train_df['房屋面积'], train_df['月租金'], c = "blue", marker = "s")
# plt.title("Looking for outliers")
# plt.xlabel("RoomArea")
# plt.ylabel("Rental")
# plt.show()

## 特征构建与模型训练

In [4]:
drop_cols=['小区房屋出租数量','时间','地铁线路'] # 去掉的特征列'小区房屋出租数量','地铁线路','时间'
# 对房屋方向进行labelEncoder
try:
    lb_encoder=LabelEncoder()
    lb_encoder.fit(train_df.loc[:,'房屋朝向'].append(test_df.loc[:,'房屋朝向'])) 
    train_df.loc[:,'房屋朝向']=lb_encoder.transform(train_df.loc[:,'房屋朝向'])
    test_df.loc[:,'房屋朝向']=lb_encoder.transform(test_df.loc[:,'房屋朝向'])
except Exception as e:
    print(e)

# 构造相对高度、房屋总数量、和房屋平均面积
#     train_df.loc[:,'相对高度']=train_df.loc[:,'楼层']/(train_df.loc[:,'总楼层']+1)
#     test_df.loc[:,'相对高度']=test_df.loc[:,'楼层']/(test_df.loc[:,'总楼层']+1)
#     train_df.loc[:,'总房间数']=train_df.loc[:,'厅的数量']+train_df.loc[:,'卧室数量']+train_df.loc[:,'卫的数量']
#     test_df.loc[:,'总房间数']=test_df.loc[:,'厅的数量']+test_df.loc[:,'卧室数量']+test_df.loc[:,'卫的数量']
#     train_df.loc[:,'Area/Room']=train_df.loc[:,'房屋面积']/(train_df.loc[:,'总房间数']+1)
#     test_df.loc[:,'Area/Room']=test_df.loc[:,'房屋面积']/(test_df.loc[:,'总房间数']+1)
train_df['相对高度']=train_df['楼层'] / (train_df['总楼层'] + 1)
test_df['相对高度']=test_df['楼层'] / (test_df['总楼层'] + 1)
train_df['总房间数']=train_df['卧室数量'] + train_df['厅的数量'] + train_df['卫的数量']
test_df['总房间数']=test_df['卧室数量'] + test_df['厅的数量'] + test_df['卫的数量']
train_df['Area/Room']=train_df['房屋面积'] / (train_df['总房间数']+1)
test_df['Area/Room']=test_df['房屋面积'] / (test_df['总房间数']+1)

# 对商圈位置、地铁线路按照租金均值进行rank并作为其label
rank_cols=['位置','地铁线路']
for col in rank_cols:
    # 对df按照当前的col特征列进行分组，求每个组的平均值，并按照'Rental'从小到大的顺序进行排序，
    rank_df=train_df.loc[:,[col,'月租金']].groupby(col,as_index=False).mean().sort_values(by='月租金').reset_index(drop=True)
    rank_df.loc[:,col+'_rank']=rank_df.index+1        # +1，为缺失值预留一个0值的rank
    rank_fe_df=rank_df.drop(['月租金'],axis=1)
    # 将rank_fe_df按照col列合并入train_df&test_df
    train_df=train_df.merge(rank_fe_df,how='left',on=col)
    test_df=test_df.merge(rank_fe_df,how='left',on=col)
    # 删除当前的col列
    try:
        train_df.drop([col],axis=1,inplace=True)
        test_df.drop([col],axis=1,inplace=True)
    except Exception as e:
        print(e)

# 删除drop_cols中的特征列
for drop_col in drop_cols:
        try:
            train_df.drop(drop_col,axis=1,inplace=True)
            test_df.drop(drop_col,axis=1,inplace=True)
        except Exception as e:
            pass

######################################################
## 1.904/0.754408后的特征测试
######################################################
#     # 卧室厅、卧室卫、厅卫
#     train_df['BedLivingroom']=train_df['厅的数量']+train_df['卧室数量']
#     test_df['BedLivingroom']=test_df['厅的数量']+test_df['卧室数量']
#     train_df['DedBathroom']=train_df['卫的数量']+train_df['卧室数量']
#     test_df['DedBathroom']=test_df['卫的数量']+test_df['卧室数量']
#     train_df['LivBathroom']=train_df['卫的数量']+train_df['厅的数量']
#     test_df['LivBathroom']=test_df['卫的数量']+test_df['厅的数量']
# 卧室占比、厅占比、卫占比
#     train_df['BedroomRitio']=train_df['卧室数量']/train_df['总房间数']
#     test_df['BedroomRitio']=test_df['卧室数量']/test_df['总房间数']
#     train_df['LivingRitio']=train_df['厅的数量']/train_df['总房间数']
#     test_df['LivingRitio']=test_df['厅的数量']/test_df['总房间数']   
#     train_df['BathRitio']=train_df['卫的数量']/train_df['总房间数']
#     test_df['BathRitio']=test_df['卫的数量']/test_df['总房间数']    
#卧室面积、厅的面积、卫的面积 
train_df['卧室面积']=train_df['房屋面积']*(train_df['卧室数量']/train_df['总房间数'])
test_df['卧室面积']=test_df['房屋面积']*(test_df['卧室数量']/test_df['总房间数'])  
train_df['厅的面积']=train_df['房屋面积']*(train_df['厅的数量']/train_df['总房间数'])
test_df['厅的面积']=test_df['房屋面积']*(test_df['厅的数量']/test_df['总房间数'])   
train_df['卫的面积']=train_df['房屋面积']*(train_df['卫的数量']/train_df['总房间数'])
test_df['卫的面积']=test_df['房屋面积']*(test_df['卫的数量']/test_df['总房间数'])
#     train_df['else']=train_df['房屋面积'] - train_df['卧室数量']
#     test_df['else']=test_df['房屋面积'] - test_df['卧室数量']
######################################################
## 1.906/0.738673后的特征测试
######################################################
#     train_df['距离'] = train_df['距离'].fillna(1.)
#     test_df['距离'] = test_df['距离'].fillna(1.)
#     def fill_na(df):
#         a=df['地铁线路_rank'].dropna().mode()
#         df['地铁线路_rank'] = df['地铁线路_rank'].fillna(a)
#         a=df['地铁站点'].dropna().mode()
#         df['地铁站点'] = df['地铁站点'].fillna(a)
#         return df
#     train_df = train_df.groupby('小区名').apply(lambda x:fill_na(x))
#     test_df = test_df.groupby('小区名').apply(lambda x:fill_na(x))
#     # 对小区、区、商圈的角度来进行构建特征
#     items = ['房屋面积','卧室数量','厅的数量','卫的数量','总楼层']
#     for item in items:
#         temp_train = train_df.groupby('小区名', as_index=False)[item].agg({item+'sum1':'sum'})
#         temp_test = test_df.groupby('小区名', as_index=False)[item].agg({item+'sum1':'sum'})
#     train_df.merge(temp_train, on='小区名', how='left')
#     test_df.merge(temp_test, on='小区名', how='left')

#     items = ['房屋面积','卧室数量','厅的数量','卫的数量','总楼层']
#     for item in items:
#         temp_train = train_df.groupby('区', as_index=False)[item].agg({item+'sum2':'sum'})
#         temp_test = test_df.groupby('区', as_index=False)[item].agg({item+'sum2':'sum'})
#     train_df.merge(temp_train, on='区', how='left')
#     test_df.merge(temp_test, on='区', how='left')

# 统计小区附近的地铁站点（效果明显）
temp = train_df.groupby('小区名')['地铁站点'].count().reset_index()
temp.columns = ['小区名','地铁站点数量']
train_df = train_df.merge(temp, how = 'left',on = '小区名')
test_df = test_df.merge(temp, how = 'left',on = '小区名')

temp1 = train_df.groupby('小区名')['地铁线路_rank'].count().reset_index()
temp1.columns = ['小区名','地铁线路数量']
train_df = train_df.merge(temp1, how = 'left',on = '小区名') 
test_df = test_df.merge(temp1, how = 'left',on = '小区名')
######################################################
## 1.87246/0.72992后的特征测试
######################################################
train_df['PerFloorBedroomArea'] = train_df['相对高度'] * train_df['卧室面积']
test_df['PerFloorBedroomArea'] = test_df['相对高度'] * test_df['卧室面积']
#     train_df['PerFloorArea'] = train_df['相对高度'] * train_df['卧室面积']
#     test_df['PerFloorArea'] = test_df['相对高度'] * test_df['卧室面积']

# 构造训练、测试集
X_train=train_df.drop(['月租金'],axis=1)
Y_train=train_df.loc[:,'月租金'].values
test_id=test_df.loc[:,'id']
X_test=test_df.drop(['id'],axis=1)

reg_model=xgb.XGBRegressor(max_depth=8, # 构建树的深度，越大越容易过拟合
                           n_estimators=3880, # 最佳迭代次数
                           learning_rate=0.1, # 学习率
                           n_jobs=-1) # 启动cpu所有核
reg_model.fit(X_train, # 训练集
              Y_train, # 训练集标签
              eval_set=[(X_train,Y_train)], # 验证集
              verbose = 10, # 训练多少轮打印一次结果
              early_stopping_rounds=10 # 每经过10轮rmse必须下降，否则停止训练
             )

y_pred=reg_model.predict(X_test)

sub_df=pd.DataFrame({
    'id':test_id,
    'price':y_pred
})
sub_df.to_csv('./result/xgb.csv',index=False)


Index(['小区名', '楼层', '总楼层', '房屋面积', '房屋朝向', '居住状态', '卧室数量', '厅的数量', '卫的数量',
       '出租方式', '区', '地铁站点', '距离', '装修情况', '月租金', '相对高度', '总房间数', 'Area/Room',
       '位置_rank', '地铁线路_rank'],
      dtype='object') Index(['id', '小区名', '楼层', '总楼层', '房屋面积', '房屋朝向', '居住状态', '卧室数量', '厅的数量',
       '卫的数量', '出租方式', '区', '地铁站点', '距离', '装修情况', '相对高度', '总房间数', 'Area/Room',
       '位置_rank', '地铁线路_rank'],
      dtype='object')
[0]	validation_0-rmse:8.87601
Will train until validation_0-rmse hasn't improved in 10 rounds.
[10]	validation_0-rmse:3.98678
[20]	validation_0-rmse:2.60276
[30]	validation_0-rmse:2.23058
[40]	validation_0-rmse:2.08331
[50]	validation_0-rmse:1.99558
[60]	validation_0-rmse:1.94068
[70]	validation_0-rmse:1.8976
[80]	validation_0-rmse:1.85802
[90]	validation_0-rmse:1.81599
[100]	validation_0-rmse:1.76781
[110]	validation_0-rmse:1.72742
[120]	validation_0-rmse:1.69263
[130]	validation_0-rmse:1.66549
[140]	validation_0-rmse:1.64268
[150]	validation_0-rmse:1.61379
[160]	validation_0-rms

[2340]	validation_0-rmse:0.670683
[2350]	validation_0-rmse:0.669638
[2360]	validation_0-rmse:0.668918
[2370]	validation_0-rmse:0.66797
[2380]	validation_0-rmse:0.667248
[2390]	validation_0-rmse:0.666174
[2400]	validation_0-rmse:0.664996
[2410]	validation_0-rmse:0.664227
[2420]	validation_0-rmse:0.663484
[2430]	validation_0-rmse:0.662789
[2440]	validation_0-rmse:0.662282
[2450]	validation_0-rmse:0.661125
[2460]	validation_0-rmse:0.66038
[2470]	validation_0-rmse:0.659436
[2480]	validation_0-rmse:0.659014
[2490]	validation_0-rmse:0.65819
[2500]	validation_0-rmse:0.657376
[2510]	validation_0-rmse:0.656999
[2520]	validation_0-rmse:0.656334
[2530]	validation_0-rmse:0.655808
[2540]	validation_0-rmse:0.655408
[2550]	validation_0-rmse:0.654607
[2560]	validation_0-rmse:0.654044
[2570]	validation_0-rmse:0.653535
[2580]	validation_0-rmse:0.652857
[2590]	validation_0-rmse:0.652238
[2600]	validation_0-rmse:0.651419
[2610]	validation_0-rmse:0.650514
[2620]	validation_0-rmse:0.650128
[2630]	validation